# Greedy algorithm

In [1]:
# Import libraries
%matplotlib inline
import matplotlib.pyplot as plt
import time
import numpy as np
from numpy import random
import math
import scipy
from scipy import stats
from random import choice
import networkx as nx
import json
from networkx.readwrite import json_graph
import line_profiler
import IPython
ip = IPython.get_ipython()
ip.define_magic('lprun', line_profiler.magic_lprun)

In [2]:
# Load network graph
with open("graph/nc_mini.json", "r") as graph_data:
    graph_data = json.load(graph_data)
    NC_digraph = json_graph.node_link_graph(graph_data)

Run Greedy Algorithm to solve |S|=3 Influence Maximization Problem for nc_mini.json graph. Use N=1,000 for the influence function.

In [3]:
def activateNodesOptGreedy(detStartNodes):

    nx.set_node_attributes(NC_digraph, 'activated', False)
    nx.set_node_attributes(NC_digraph, 'explored', False)
    
    activated = 0
    nodes = []
    
    for n in detStartNodes:
        nodes.append(n)
        NC_digraph.node[n]['activated'] = True
        activated = activated + 1
    
    start = nodes[0]

    while len(nodes)>0:

        startNode = nodes[0]

        if NC_digraph.node[startNode]['explored']==False:

            NC_digraph.node[startNode]['explored'] = True

            successors = []

            for succNode in NC_digraph.succ[startNode]:

                if NC_digraph.node[succNode]['activated']==False:

                    alpha = NC_digraph[startNode][succNode]['weight']
                    beta = NC_digraph.node[succNode]['review_count']

                    randUnif = random.uniform(0,1)
                    randBeta = np.sqrt(random.beta(alpha, beta))

                    if randUnif < randBeta:
                        NC_digraph.node[succNode]['activated'] = True
                        successors.append(succNode)
                        activated = activated + 1

        nodes = nodes[1:]
        nodes = nodes + successors

    return activated

def activateNodesLoopGreedy(N, startNodes):
    
    random.seed()
    searchNodes = NC_digraph.nodes()
    actNodeOpt = []
    
    for s in range(startNodes):
        maxActNode = 0
        maxActNodeStart = []

        for n in searchNodes:
            detStartNodes = actNodeOpt + [n]
            result = []
            for r in xrange(N):
                result.append(float(activateNodesOptGreedy(detStartNodes)))
            act = np.mean(result)
            if act>maxActNode:
                maxActNode = act
                maxActNodeStart = n
            
        searchNodes.remove(maxActNodeStart)
        actNodeOpt.append(maxActNodeStart)
    
    return actNodeOpt, maxActNode
            

Try running the greedy algorithm a few times...

In [4]:
%time solNodes, solActivated = activateNodesLoopGreedy(1000, 3)

CPU times: user 7min 56s, sys: 2.34 s, total: 7min 58s
Wall time: 8min 3s


In [5]:
solNodes, solActivated = activateNodesLoopGreedy(1000, 3)
print 'Optimal nodes: %s\nAverage of %d nodes activated' % (solNodes, solActivated)

Optimal nodes: [u'NzWLMPvbEval0OVg_YDn4g', u'VhI6xyylcAxi0wOy2HOX3w', u'ts7EG6Zv2zdMDg29nyqGfA']
Average of 46 nodes activated
